# 1 Introduction
First, we have to activate the environment. The first run may take a while as external libraries will be downloaded and compiled.

In [1]:
Pkg.activate(".")

  Activating project at `~/code/OpenScience21.jl`


# 2. The Algorithm
In this notebook, I will describe the training process on a real device and a noisy simulator. Technically the algorithm is the same as in the notebook [04_algorithm_evaluation](04_algorithm_evaluation.ipynb) except for the use of the error mitigation technic which was not necessary on the simulator written in Julia. 

We will use here the codes described in the notebooks:
* [01_solution](01_solution.ipynb)
* [03_trotter_step](03_trotter_step.ipynb)
* [04_algorithm_evaluation](04_algorithm_evaluation.ipynb)
Therefore in the beginning we import the script with includes all necessary functions.

In [2]:
# The init file contains useful scripts and imports as well as connections to the IBM Qiskit account.
include("src/Challenge/init.jl")

evaluate (generic function with 1 method)

First, we have to find the parameter for the U4 gate in trotter steps (see [03_trotter_step](03_trotter_step.ipynb)), at the same time we generate the whole circuit with ten trotter steps, this is our goal to achieve.

In [3]:
t = π
trotter_steps = 10
qc_full, params, params2, params3 = generate_circuit(trotter_steps, trotter_steps, t, init=true)
addMeasuresOS(qc_full)
# Execute simulation
check_simulation_err(qc_full, t)
qc_full

                                                                   »
 q_0: ─────────────────────────────────────────────────────────────»
      ┌─────────────────────────┐                             ┌───┐»
 q_1: ┤ U3(3.1418,4.205,3.4689) ├─────────────────────────────┤ X ├»
      └─────────────────────────┘                             └─┬─┘»
 q_2: ──────────────────────────────────────────────────────────┼──»
                 ┌───┐           ┌───────────────────────────┐  │  »
 q_3: ───────────┤ X ├───────────┤ U3(3.1409,-0.72181,6.396) ├──■──»
                 └───┘           └───────────────────────────┘     »
 q_4: ─────────────────────────────────────────────────────────────»
                 ┌───┐           ┌───────────────────────────┐     »
 q_5: ───────────┤ X ├───────────┤ U3(-3.1416,6.6735,3.4094) ├─────»
                 └───┘           └───────────────────────────┘     »
 q_6: ─────────────────────────────────────────────────────────────»
                                  

As was described in [01_solution](01_solution.ipynb) we have to iterate 5 times the QML algorithm to find the best ansact parameter. So let's begin :)

# 3. Step 1

In [4]:
const st = 2
step1_check_qc, _, _, _ = generate_circuit(trotter_steps, 8, t, params, params2, params3, init=false)
step1_step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=true)
step1_step_qc_exp

                                                                               »
q_0: ──────────────────────────────────────────────────────────────────────────»
     ┌─────────────────────────┐                             ┌───┐ ┌──────────┐»
q_1: ┤ U3(3.1418,4.205,3.4689) ├─────────────────────────────┤ X ├─┤ Rz(3π/5) ├»
     └─────────────────────────┘                             └─┬─┘ └──────────┘»
q_2: ──────────────────────────────────────────────────────────┼───────────────»
                ┌───┐           ┌───────────────────────────┐  │  ┌───────────┐»
q_3: ───────────┤ X ├───────────┤ U3(3.1409,-0.72181,6.396) ├──■──┤ Ry(-3π/5) ├»
                └───┘           └───────────────────────────┘     └───────────┘»
q_4: ──────────────────────────────────────────────────────────────────────────»
                ┌───┐           ┌───────────────────────────┐                  »
q_5: ───────────┤ X ├───────────┤ U3(-3.1416,6.6735,3.4094) ├──────────────────»
                └───┘       

We have to define our step ansact.

In [5]:
step1_ansact = generate_ansact()

                                                                   »
q_0: ──────────────────────────────────────────────────────────────»
     ┌──────────────────────────┐                                  »
q_1: ┤ U3(5.6622,5.5962,5.5082) ├───────────────────■──────────────»
     └──────────────────────────┘                   │              »
q_2: ───────────────────────────────────────────────┼──────────────»
     ┌──────────────────────────┐                 ┌─┴─┐            »
q_3: ┤ U3(5.6622,5.5962,5.5082) ├──■──────────────┤ X ├────────────»
     └──────────────────────────┘  │              └───┘            »
q_4: ──────────────────────────────┼───────────────────────────────»
     ┌──────────────────────────┐┌─┴─┐┌───────────────────────────┐»
q_5: ┤ U3(5.6622,5.5962,5.5082) ├┤ X ├┤ U3(5.6578,0.52542,3.6589) ├»
     └──────────────────────────┘└───┘└───────────────────────────┘»
q_6: ──────────────────────────────────────────────────────────────»
                                  

In the definition of the start parameter, we can do this twofold, first, we can rand it, the second, we can use the previously found parameter (for instance if we break the optimization process).

In [6]:
#step1_start_params = [6.490463894513841, 3.434943430067894, 5.375354561541277, 4.7906997798464115, 2.849325024133124, 2.7673620360366824, 1.4940499436354768, 4.487752591524359, 3.2948988322626382, 2.0225225509500486, -1.2343170007923738, 1.0431091490261049, 1.8837296276360387, 1.2169677999557502, 5.940807695305212, 5.421210358780618, 0.3173688878579456, -6.965567540712155, 4.835967484868803, 3.0127993733543517, 4.4620544822135475, 1.378337396849606, 1.980959799643362, 3.6789144092845376, 5.417860886313657, 1.8687391611628605, 5.092938820960128]
step1_start_params = getRandParameters(step1_ansact)

27-element Vector{Number}:
 3.174765060739268
 1.2352964942043025
 5.370536393616918
 0.705716489668319
 2.133791325920609
 1.8623490353547987
 6.241460948646937
 1.8113874632961866
 1.736619041647653
 1.2538575749674408
 5.229545524062075
 0.6117270384484857
 5.49583290191272
 ⋮
 1.2472044044411106
 3.0972664486330737
 5.841915388639033
 2.717644770610637
 0.9191480391472905
 1.6433227146089178
 6.163900461755911
 3.9251981135655485
 3.2714857434501283
 5.532264362451688
 1.0588367779623873
 5.697330532533117

In [7]:
# Set parameters
setparameters!(step1_ansact, step1_start_params)

# Generate step qc
step1_qc = generate_empty_circuit()
# append
append!(step1_qc, step1_step_qc_exp)
append!(step1_qc, step1_ansact)
# Add measures
addMeasuresOS(step1_qc)
step1_qc

                                                                   »
 q_0: ─────────────────────────────────────────────────────────────»
      ┌─────────────────────────┐                             ┌───┐»
 q_1: ┤ U3(3.1418,4.205,3.4689) ├─────────────────────────────┤ X ├»
      └─────────────────────────┘                             └─┬─┘»
 q_2: ──────────────────────────────────────────────────────────┼──»
                 ┌───┐           ┌───────────────────────────┐  │  »
 q_3: ───────────┤ X ├───────────┤ U3(3.1409,-0.72181,6.396) ├──■──»
                 └───┘           └───────────────────────────┘     »
 q_4: ─────────────────────────────────────────────────────────────»
                 ┌───┐           ┌───────────────────────────┐     »
 q_5: ───────────┤ X ├───────────┤ U3(-3.1416,6.6735,3.4094) ├─────»
                 └───┘           └───────────────────────────┘     »
 q_6: ─────────────────────────────────────────────────────────────»
                                  

In [8]:
@enum ExecutePlace NoisySim RealDevice JuliaSim
#useExecutePlace = NoisySim
useExecutePlace = JuliaSim

JuliaSim::ExecutePlace = 2

In [9]:
if useExecutePlace == JuliaSim   
    step1_loss(params) = loss_expected_zero_state(execute(backend, step1_qc, params))
    step1_dloss(params) = real(step1_loss'(params))
    step1_of = OptimizationFunction(false, (x) -> (step1_loss(x), step1_dloss(x)), step1_loss)
else
    if useExecutePlace == NoisySim
        useBackend = qiskitBackendNoisySim
    else 
        useBackend = qiskitBackendJakarta
    end
    
    step1_of = OptimizationFunction(
            false,
            (x) -> qderivative(useBackend, step1_qc, loss_expected_zero_state, x),
            (x) -> loss_expected_zero_state(execute(useBackend, setAndConvert(step1_qc, x))))
end

OptimizationFunction(false, var"#5#8"(), step1_loss)

In [10]:
val, xparams, itr = gradientDescent(step1_of, step1_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

3.174765060739268, 1.2352964942043025, 5.370536393616918, 0.705716489668319, 2.133791325920609, 1.8623490353547987, 6.241460948646937, 1.8113874632961866, 1.736619041647653, 1.2538575749674408, 5.229545524062075, 0.6117270384484857, 5.49583290191272, 1.2555709235282333, 5.035806231226277, 1.2472044044411106, 3.0972664486330737, 5.841915388639033, 2.717644770610637, 0.9191480391472905, 1.6433227146089178, 6.163900461755911, 3.9251981135655485, 3.2714857434501283, 5.532264362451688, 1.0588367779623873, 5.697330532533117
Start iteration 1, value: 1.8354221975150546, best: 1.8354221975150546, dval: 0.9441119197378129
3.167034258388486, 1.232706902538618, 5.373274936187719, 0.710583315617476, 2.134170376158936, 1.8567605898711355, 6.232931733924895, 1.8142268713086875, 1.7394707846208979, 1.2527030629688367, 5.231315722906861, 0.6091374467828011, 5.500257344308396, 1.257136198689822, 5.034470292574164, 1.2566455236384888, 3.0972664486330737, 5.843685587483819, 2.725717869191087, 0.916973291

Start iteration 20, value: 0.1857627910138712, best: 0.1857627910138712, dval: 0.060227296861851645
2.0533893711366678, 1.0876650435815625, 5.999776290838898, 1.6538504420325286, 2.1343895254527463, 0.479039769745589, 7.0673916992513695, 7.197013440954411, 2.275696278951528, 1.2459537204689273, 5.580474872993118, 0.4640955878257493, 6.6798377955537545, 1.297707837852142, 5.230336290224326, 1.3814838517770898, 3.0972664486330737, 6.192844737570084, 2.903770300707955, 0.8055416448656105, 1.774785199201683, 6.04612050349286, 3.9251981135655485, 3.3058093000257656, 5.647117113017518, 1.0588367779623873, 6.052867285542884
Start iteration 21, value: 0.1848246938401448, best: 0.1848246938401448, dval: 0.0559241510672973
1.9883159876817196, 1.0876642010025648, 6.00259205191984, 1.6475125261762087, 2.1344379582204644, 0.4708625734695939, 7.0677204625328445, 7.19641300934714, 2.2757621638105974, 1.2459539714630719, 5.590524035368478, 0.4640947452467517, 6.658711305322146, 1.297693033957696, 5.23

Start iteration 38, value: 0.15389718796704097, best: 0.15389718796704097, dval: 0.044373037073211746
1.8724536297136032, 1.0876482376063112, 6.180803904921589, 1.7442012432744156, 2.153213027446231, 0.40582137337417085, 7.200078010272258, 7.011257191687332, 2.275225544109677, 1.2459549212897925, 5.5429704937340745, 0.4640787818504968, 6.562120930236664, 1.395693016155641, 5.229811041252649, 1.1009908519844536, 3.0972664486330737, 6.155340358311035, 2.9297240925660173, 0.7835482245598417, 1.7804403743404928, 5.937929726973052, 3.9251981135655485, 2.5209591612867914, 5.5617955712071865, 1.0588367779623873, 6.161987445897997
Start iteration 39, value: 0.15337812759241992, best: 0.15337812759241992, dval: 0.04813146184119249
1.86821534821282, 1.0876482299691346, 6.199671285845564, 1.7444743464467358, 2.1532124796364553, 0.40980456220482386, 7.176936121192759, 6.999698792643905, 2.275199036891744, 1.2459549260522855, 5.528255474791207, 0.46407877421331994, 6.563886224079339, 1.389370988778

Start iteration 56, value: 0.17284661021928754, best: 0.11354434925756439, dval: 0.2893179704715644
Big value increase: val: 0.17284661021928754, e.val: 0.11354434925756439, max(α): 19.565185853913558
1.7121059559129503, 1.0883935148971984, 6.322525005705309, 1.6709095478955358, 2.1974921926611324, 0.4138729306555232, 7.382661982007056, 6.891259379483155, 2.2096136400342052, 1.2459551928200387, 4.868615858807119, 0.5089565201779275, 6.412562859368405, 2.1181770997748455, 5.670561551945082, 1.070667752451031, 3.0972664486330737, 5.480985723384095, 2.9242358255159, 0.8562595084018, 1.7386023532664472, 5.804032578183428, 3.9251981135655485, 1.5394741797938623, 5.753015111110594, 1.0588367779623873, 6.183956019621896
Start iteration 57, value: 0.1016971534549821, best: 0.1016971534549821, dval: 0.06413178227287117
1.683274151322147, 1.0886026378534515, 6.321894431541664, 1.6543455099633813, 2.211608393273567, 0.4138626671551219, 7.356754576229722, 7.086005166435919, 2.246208334279021, 1.24

Start iteration 73, value: 0.04474627758521203, best: 0.04474627758521203, dval: 0.0700718962040647
1.5647950039834482, 1.0915676311511864, 6.3160370102259, 1.5288082093095825, 2.6166535373189057, 0.5377331224914302, 7.15376243116805, 7.240990323179875, 2.119731795952705, 1.2459553907475556, 4.793364324704328, 0.6989333975347051, 6.373228096405077, 2.121829642745208, 5.772852409847594, 1.234701671948175, 3.0972664486330737, 5.405734189281305, 2.9240499471567656, 0.8717897482517101, 1.7346207293375666, 5.559621115171301, 3.9251981135655485, 1.5620885624383583, 5.876962113922496, 1.0588367779623873, 6.1754195964648835
Start iteration 74, value: 0.04216509842791858, best: 0.04216509842791858, dval: 0.05056747527048309
1.5646454711977864, 1.0915362046241868, 6.315981799413899, 1.525669743014152, 2.6315595585017313, 0.6407127104921208, 7.151569102974677, 7.302582688015618, 2.1197339378253606, 1.2459558265193684, 4.7884379591494755, 0.6970524602944225, 6.375317453806416, 2.12187444782902, 5.

1.5635100853121024, 1.0917118236276133, 6.3159948222499, 1.4711991059070775, 2.864722778251968, 0.5910903335776457, 7.078286061183313, 7.566781683617283, 2.1197337264332745, 1.2763161662141713, 4.700447748069464, 0.7033184888975024, 6.357479259616941, 2.1238282986034744, 5.895526846826596, 1.2888038182060388, 3.0972664486330737, 5.312817612646088, 2.847970632820437, 0.6590430843398853, 1.746339104328976, 5.457345288602492, 3.9251981135655485, 1.5581135603995695, -0.3211703519672811, 1.0588367779623873, 6.115714952831865
Start iteration 95, value: 0.015393751102921743, best: 0.015393751102921743, dval: 0.03188700652326093
Start iteration 96, value: 0.018993242658681842, best: 0.015393751102921743, dval: 0.07617543851188902
Big value increase: val: 0.018993242658681842, e.val: 0.015393751102921743, max(α): 7.51265429615535
1.563510001900999, 1.0917118309770102, 6.315994822261358, 1.4740835803859922, 2.8856690288036475, 0.6490551935724749, 7.078000863681776, 7.567459861702807, 2.119716208

(0.008532872893815812, Number[1.5635092417642888, 1.0917132379979078, 6.31599482226065, 1.4732541710377316, 2.9481921658480417, 0.6504783088529388, 7.078068461733884, 7.652359579633591, 2.1197203640629665, 1.3230549362252126  …  5.297841636161047, 2.8424779967592784, 0.5804122410190855, 1.7463988467786122, 5.45851535917415, 3.9251981135655485, 1.5532588296646457, -0.2882992641283684, 1.0588367779623873, 6.115740790431033], 100)

Is worth seeing the code of qderivative function available in QuantumCircuits library: https://github.com/Adgnitio/QuantumCircuits.jl/blob/main/src/Execute/Execute.jl

This method calculates the loss function, and the derivative of the loss function using the parameter shift rule (_"Quantum Circuit Learning"_ https://arxiv.org/abs/1803.00745, _"Evaluating analytic gradients on quantum hardware"_ https://arxiv.org/abs/1811.11184) and additionally, it performs the measured error mitigation as described in the notebook [01_solution](01_solution.ipynb). The _Zero Noise Extrapolation_ method is not included because I haven't known about it during the implementation but I'm going to add it in the future.

The parameter for step 1 was found on the **real device**. The code used in training can be found in [sol_step1](src/Challenge/steps/sol_step1.jl)(the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a real device can be found in [sol_res_step1_Jakarta_1](src/Challenge/steps/sol_res_step1_Jakarta_1.txt) and [sol_res_step1_Jakarta_2](src/Challenge/steps/sol_res_step1_Jakarta_2.txt).

It is worth noting that the printing of the solution can be used to stop the current optimization process and start it again from the previously founded the best solution. This is not necessary for the Julia simulator but in the real device (and even on the simulator) some noises can impact the calculation of derivatives. If we can execute gradient descent many times this doesn't matter because finally, gradient descent finds the proper direction but on the real device, we would like to have as few circuits run as possible.

# 4. Step 2
